### Modelo de aprendizaje automático

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación. Para ello, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno. 

In [1]:
# Importando librerias

import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#Traigo la tabla para realizar el modelo
modelo_item= pd.read_csv('data/modelo_item.csv', encoding='utf8')
modelo_item['item_id']=modelo_item['item_id'].astype('string')
modelo_item.head()

,item_id,app_name,genres
0,719000,Strata Spaces VR – Professional Edition Upgrade,Design &amp; Illustration
1,719000,Strata Spaces VR – Professional Edition Upgrade,Animation &amp; Modeling
2,292180,X-Plane 10 Global - 64 Bit,Simulation
3,358040,HTC Vive,Sin Datos
4,629796,EVE Online: 15400 PLEX,RPG


Para obtener el modelo se deben convertir en dummies todos los generos disponibles

In [3]:
modelo_item_2= pd.get_dummies(modelo_item, columns=["genres"], prefix="")

In [4]:
modelo_item_2.head(3)

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_RPG,_Racing,_Simulation,_Sin Datos,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,719000,Strata Spaces VR – Professional Edition Upgrade,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,719000,Strata Spaces VR – Professional Edition Upgrade,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,292180,X-Plane 10 Global - 64 Bit,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [5]:
modelo_item_3= modelo_item_2.groupby(["item_id","app_name"]).sum().reset_index()
modelo_item_3.head(4)

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_RPG,_Racing,_Simulation,_Sin Datos,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1002,Rag Doll Kung Fu,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100400,Silo 2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10090,Call of Duty: World at War,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
modelo_item_3.shape

(32132, 25)

In [8]:
modelo_item_3['item_id']=modelo_item_3['item_id'].astype('string')

In [30]:
modelo_item_3['item_id']=modelo_item_3['item_id'].astype('int')

In [31]:
modelo_item_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32132 entries, 0 to 32131
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   item_id                     32132 non-null  int32 
 1   app_name                    32132 non-null  object
 2   _Accounting                 32132 non-null  uint8 
 3   _Action                     32132 non-null  uint8 
 4   _Adventure                  32132 non-null  uint8 
 5   _Animation &amp; Modeling   32132 non-null  uint8 
 6   _Audio Production           32132 non-null  uint8 
 7   _Casual                     32132 non-null  uint8 
 8   _Design &amp; Illustration  32132 non-null  uint8 
 9   _Early Access               32132 non-null  uint8 
 10  _Education                  32132 non-null  uint8 
 11  _Free to Play               32132 non-null  uint8 
 12  _Indie                      32132 non-null  uint8 
 13  _Massively Multiplayer      32132 non-null  ui

Aplicar modelo coseno

In [25]:
similitudes = cosine_similarity(modelo_item_3.iloc[:,3:])

In [26]:
similitudes

array([[1.        , 0.        , 0.        , ..., 0.40824829, 0.        ,
        0.70710678],
       [0.        , 1.        , 0.        , ..., 0.40824829, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.40824829, 0.40824829, 0.        , ..., 1.        , 0.54772256,
        0.57735027],
       [0.        , 0.        , 0.        , ..., 0.54772256, 1.        ,
        0.31622777],
       [0.70710678, 0.        , 0.        , ..., 0.57735027, 0.31622777,
        1.        ]])

In [34]:
def recomendacion_juego(item_id):
    
    # Filtrar el juego e igualarlo a  su ID
    juego_seleccionado = modelo_item_3[modelo_item_3['item_id'] == item_id]
    # devolver error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calcular la matriz de similitud coseno
    similitudes = cosine_similarity(modelo_item_3.iloc[:,3:])
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo_item_3[modelo_item_3['item_id'] == item_id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item_3.iloc[indices_juegos_similares]['app_name']
    
    return {
        "Juegos_recomendados" :juegos_recomendados}

In [35]:
recomendacion_juego(719000)

{'Juegos_recomendados': 29678                 imos LOFT
 13985                  Aseprite
 17235            RealityCapture
 15582    Qubicle Utility Module
 15581       Qubicle Mesh Module
 Name: app_name, dtype: object}

In [32]:
modelo_final_csv = "modelo_final.csv"
modelo_item_3.to_csv(modelo_final_csv , index=False, encoding="utf-8")

In [33]:
funcion_6= pd.read_csv("modelo_final.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/funcion_6.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(funcion_6)
pq.write_table(table,output_file)

In [14]:
#No me corre en render, pruebo achicar la funcion
#Cuento filas
cant_filas= len(modelo_item_3)
#Calculo la mitad
mitad_filas= cant_filas // 10
#Selecciono la mitad superior
modelo_render= modelo_item_3.iloc[:mitad_filas]


In [15]:
modelo_render.shape

(3213, 25)

In [16]:
modelo_render.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3213 entries, 0 to 3212
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   item_id                     3213 non-null   int32 
 1   app_name                    3213 non-null   object
 2   _Accounting                 3213 non-null   uint8 
 3   _Action                     3213 non-null   uint8 
 4   _Adventure                  3213 non-null   uint8 
 5   _Animation &amp; Modeling   3213 non-null   uint8 
 6   _Audio Production           3213 non-null   uint8 
 7   _Casual                     3213 non-null   uint8 
 8   _Design &amp; Illustration  3213 non-null   uint8 
 9   _Early Access               3213 non-null   uint8 
 10  _Education                  3213 non-null   uint8 
 11  _Free to Play               3213 non-null   uint8 
 12  _Indie                      3213 non-null   uint8 
 13  _Massively Multiplayer      3213 non-null   uint

In [30]:
similitudes_render = cosine_similarity(modelo_render.iloc[:,3:])

In [27]:
modelo_render.head()

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_RPG,_Racing,_Simulation,_Sin Datos,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1002,Rag Doll Kung Fu,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100400,Silo 2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10090,Call of Duty: World at War,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,100980,3D-Coat V4.8,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
modelo_render.to_parquet("data/modelo_render.parquet")

In [18]:
def encontrar_juegos_similares(item_id):
    # Encuentra el índice del juego ingresado por ID
    juego_indice = modelo_render.index[modelo_render['item_id'] == item_id].tolist()[0]
    
    # Extrae las características del juego ingresado
    juego_caracteristicas = modelo_render.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    # Calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_render = cosine_similarity(modelo_render.iloc[:, 3:], juego_caracteristicas)
    
    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_render.argsort(axis=0)[::-1][1:6]
    indices_juegos_similares = indices_juegos_similares.flatten()[1:]
    
    # Obtiene los juegos más similares en función de los índices
    juegos_similares = modelo_render.iloc[indices_juegos_similares]['app_name']
    
    return juegos_similares


In [19]:
encontrar_juegos_similares(10)

994     Street Fighter X Tekken: Heihachi (Swap Costume)
993         Street Fighter X Tekken: Lili (Swap Costume)
320                   INSURGENCY: Modern Infantry Combat
2756                                          Shadow Man
Name: app_name, dtype: object

In [24]:
def recomendacion_juego(item_id):
    
    game = modelo_render[modelo_render['item_id'] == item_id]
    
    if game.empty:
        return("El juego '{item_id}' no posee registros.")
    
    # Obtiene el índice del juego dado
    idx = game.index[0]

    # Toma una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Define el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_render.sample(n=sample_size, random_state=42)  # Ajusta la semilla aleatoria según sea necesario

    # Calcula la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_render.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtiene las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordena los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtiene los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Lista de juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [28]:
recomendacion_juego(10)

{'similar_games': ['Sacred Citadel: Jungle Hunt',
  'Gotham City Impostors Free to Play: Office Bat',
  'Dogfight 1942 Fire Over Africa',
  'Choplifter HD',
  'Painkiller Hell & Damnation: Satan Claus DLC']}

In [29]:
modelo_render.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3213 entries, 0 to 3212
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   item_id                     3213 non-null   int32 
 1   app_name                    3213 non-null   object
 2   _Accounting                 3213 non-null   uint8 
 3   _Action                     3213 non-null   uint8 
 4   _Adventure                  3213 non-null   uint8 
 5   _Animation &amp; Modeling   3213 non-null   uint8 
 6   _Audio Production           3213 non-null   uint8 
 7   _Casual                     3213 non-null   uint8 
 8   _Design &amp; Illustration  3213 non-null   uint8 
 9   _Early Access               3213 non-null   uint8 
 10  _Education                  3213 non-null   uint8 
 11  _Free to Play               3213 non-null   uint8 
 12  _Indie                      3213 non-null   uint8 
 13  _Massively Multiplayer      3213 non-null   uint